In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import calendar

In [ ]:
weather_info = pd.read_csv('input/london-weather-data/london_weather.csv')
print(weather_info)

print(weather_info.shape)
print(weather_info.head)



In [ ]:
weather_info.info()
weather_info.describe().T



In [ ]:
weather_info['year'] = weather_info['date'].astype(str).str[:4]
weather_info['month'] = weather_info['date'].astype(str).str[4:6]
weather_info['date'] = weather_info['date'].astype(str).str[6:]

weather_info[['date', 'month', 'year']] = weather_info[['date', 'month', 'year']].astype(int)
weather_features = weather_info.select_dtypes(exclude = 'object').columns



In [ ]:
for x in weather_features:
  print(x, len(weather_info[x].unique())) 
london_weather_monthly_df = weather_info.groupby(['year', 'month'], as_index = False)[weather_features].mean()
# london_weather_monthly_df.plot(x = 'date', y = ['min_temp'], kind = 'line', figsize = (20,10))
london_weather_yearly_df = london_weather_monthly_df.groupby(['year'], as_index = False)[['max_temp', 'mean_temp', 'min_temp', 'global_radiation']].mean()



In [ ]:
layout = go.Layout(
    title = "Temperature of London over the years",
    plot_bgcolor = "#FFF",
    hovermode = "x",
    hoverdistance = 100,
    spikedistance = 1000,
    xaxis=dict(
        title = "Years (1979 to 2020)",
        linecolor = "dimgray",
        showgrid = False,
        showspikes = True,
        spikethickness = 1,
        spikedash = 'dash',
        spikecolor = 'darkgrey',
        spikemode = 'across',
        mirror = True
    ),
    yaxis=dict(
        title = "Temperature (°C)",
        linecolor = "dimgray",
        mirror = True
    )
)

plot = go.Figure(data = [go.Scatter(
    x = london_weather_yearly_df.year,
    y = london_weather_yearly_df.max_temp,
    name = 'Maximum',
    mode = 'lines+markers',
    line_color = 'maroon'),
    go.Scatter(
    x = london_weather_yearly_df.year,
    y = london_weather_yearly_df.min_temp,
    name = 'Minimum',
    mode = 'lines+markers',
    line_color = 'darkblue')
    ],
    layout = layout)
plot.show() 
